<a href="https://colab.research.google.com/github/LuanDavid7/AI_EKT/blob/main/AI_Answer_EKT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**CÓDIGO ATUAL**

In [ ]:
import requests
from datetime import datetime
import json

# Configurações da API Groq
API_KEY = "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"
MODEL = "llama3-70b-8192"
GROQ_URL = "https://api.groq.com/openai/v1/chat/completions"

# Tabela de categorização de problemas (simplificada para o contexto)
PROBLEMS_TABLE = {
    "emergencia_medica": {
        "descricao": "Falta de energia afetando equipamentos de suporte à vida",
        "criticidade": "Emergência",
        "acao": "Atendimento imediato com prioridade máxima"
    },
    "falta_energia": {
        "descricao": "Casa do cliente totalmente sem energia",
        "criticidade": "Médio",
        "acao": "Inspeção técnica e correção"
    },
    "falta_energia_area": {
        "descricao": "Interrupção total de energia em uma área ampla",
        "criticidade": "Alto",
        "acao": "Manutenção corretiva urgente"
    },
    "falta_fase": {
        "descricao": "Falta de uma das fases da rede elétrica",
        "criticidade": "Médio",
        "acao": "Manutenção corretiva"
    },
    "tensao_alta": {
        "descricao": "Voltagem acima do normal",
        "criticidade": "Alto",
        "acao": "Inspeção técnica urgente"
    },
    "tensao_baixa": {
        "descricao": "Voltagem abaixo do normal",
        "criticidade": "Médio",
        "acao": "Inspeção técnica"
    },
    "choque": {
        "descricao": "Cliente relata choque ao tocar em instalações",
        "criticidade": "Emergência",
        "acao": "Inspeção técnica imediata"
    },
    "fio_partido": {
        "descricao": "Cabo rompido na rede",
        "criticidade": "Emergência",
        "acao": "Isolamento imediato da área e reparo urgente"
    },
    "fogo_rede": {
        "descricao": "Fio apresentando faíscas ou princípio de incêndio",
        "criticidade": "Emergência",
        "acao": "Atendimento imediato"
    }
}

def get_current_greeting():
    """Retorna a saudação de acordo com o horário atual"""
    hour = datetime.now().hour
    if 5 <= hour < 12:
        return "Bom dia"
    elif 12 <= hour < 18:
        return "Boa tarde"
    else:
        return "Boa noite"

def generate_farewell():
    """Retorna a mensagem de encerramento padrão"""
    return """
Agradecemos seu contato! Sempre que precisar estamos disponíveis nos nossos canais:
💻 Site: https://www.neoenergia.com/web/sp

📱 Aplicativo:
Android → https://bit.ly/NELK-Android
IOS → https://bit.ly/IOS-NELK

📞 Teleatendimento: 116
Deficiente auditivo ou de fala | 0800 701 01 55
Atendimento fora da área de concessão | 0800 276 0116

Conte sempre com a gente!
"""

def analyze_problem(client_message):
    """Analisa a mensagem do cliente para identificar o problema"""
    message_lower = client_message.lower()

    # Verifica problemas críticos primeiro
    if any(term in message_lower for term in ["equipamento de suporte à vida", "respirador", "máquina de oxigênio"]):
        return "emergencia_medica"
    elif any(term in message_lower for term in ["fio partido", "cabo quebrado", "cabo rompido"]):
        return "fio_partido"
    elif any(term in message_lower for term in ["fogo no fio", "faísca", "fogo na rede", "fiação pegando fogo"]):
        return "fogo_rede"
    elif any(term in message_lower for term in ["tomando choque", "levando choque", "eletrocutando"]):
        return "choque"
    elif any(term in message_lower for term in ["tensão alta", "luz muito forte", "aparelhos queimando"]):
        return "tensao_alta"
    elif any(term in message_lower for term in ["tensão baixa", "luz fraca", "aparelhos não funcionam direito"]):
        return "tensao_baixa"
    elif any(term in message_lower for term in ["falta fase", "algumas lâmpadas acendem", "só um lado tem luz"]):
        return "falta_fase"
    elif any(term in message_lower for term in ["vizinhança sem luz", "rua inteira sem energia", "apagão"]):
        return "falta_energia_area"
    elif any(term in message_lower for term in ["sem energia", "sem luz", "queda de energia"]):
        return "falta_energia"
    else:
        return None

def check_internal_problem(client_message):
    """Verifica se o problema pode ser interno"""
    message_lower = client_message.lower()
    internal_terms = [
        "só na minha casa", "apenas aqui", "vizinhos têm luz",
        "disjuntor caiu", "disjuntor desarmou", "fusível queimou",
        "queda interna", "problema no quadro"
    ]
    return any(term in message_lower for term in internal_terms)

def generate_response(client_message, problem_type=None):
    """Gera uma resposta apropriada baseada no tipo de problema"""
    if problem_type is None:
        problem_type = analyze_problem(client_message)

    greeting = get_current_greeting()

    if problem_type is None:
        # Se não identificou um problema específico
        return f"{greeting}! Para melhor atendê-lo, poderia descrever com mais detalhes o problema que está enfrentando com a energia elétrica?"

    problem_info = PROBLEMS_TABLE.get(problem_type, {})

    if check_internal_problem(client_message):
        return f"{greeting}! Pelo que descreveu, parece ser um problema na instalação interna. Recomendo verificar:\n1. Se o disjuntor principal não desarmou\n2. Se há fusíveis queimados\n3. Se os disjuntores individuais estão ativados\nCaso persista, pode ser necessário um eletricista qualificado."

    if problem_type in PROBLEMS_TABLE:
        if problem_info["criticidade"] == "Emergência":
            # Gerar ticket de emergência
            ticket_id = f"EMG-{datetime.now().strftime('%Y%m%d%H%M%S')}"
            return f"{greeting}! Identificamos uma situação de emergência. Já estamos acionando nossa equipe técnica com prioridade máxima. Seu protocolo é {ticket_id}. Por segurança, mantenha distância do local e evite tocar em qualquer parte elétrica."
        else:
            # Gerar ticket normal
            ticket_id = f"CHM-{datetime.now().strftime('%Y%m%d%H%M%S')}"
            return f"{greeting}! Entendemos o problema e já estamos registrando sua ocorrência sob o protocolo {ticket_id}. Nossa equipe será acionada para resolver o mais breve possível."
    else:
        return f"{greeting}! Vamos analisar seu relato e encontrar a melhor solução. Poderia confirmar se o problema afeta apenas sua residência ou toda a vizinhança?"

def call_groq_api(client_message, conversation_history=[]):
    """Chama a API do Groq para gerar uma resposta mais contextualizada"""
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }

    # Construir o prompt com o contexto do atendimento
    prompt = f"""
Você é um assistente virtual da Neoenergia Elektro, uma distribuidora de energia elétrica.
Seu papel é atender clientes com empatia e eficiência, identificando problemas elétricos e orientando sobre soluções.

Contexto:
- Analise a mensagem do cliente para identificar o tipo de problema
- Se for um problema interno (da instalação do cliente), oriente sobre como verificar disjuntores e fusíveis
- Se for um problema externo (da rede elétrica), gere um ticket de atendimento
- Se for emergência (equipamento médico, fio partido, choque elétrico), priorize o atendimento
- Seja breve, objetivo e empático
- A mensagem deve ter no máximo 2 parágrafos curtos

Histórico da conversa:
{conversation_history}

Mensagem do cliente: {client_message}
"""

    data = {
        "model": MODEL,
        "messages": [
            {"role": "system", "content": prompt},
            {"role": "user", "content": client_message}
        ],
        "temperature": 0.7,
        "max_tokens": 150
    }

    try:
        response = requests.post(GROQ_URL, headers=headers, json=data)
        response.raise_for_status()
        return response.json()["choices"][0]["message"]["content"]
    except Exception as e:
        print(f"Erro ao chamar API Groq: {e}")
        return generate_response(client_message)

def main():
    """Função principal para simular o atendimento"""
    print(f"{get_current_greeting()}! Sou o assistente virtual da Neoenergia Elektro. Como posso ajudar você hoje?")

    conversation_history = []

    while True:
        client_input = input("Cliente: ").strip()

        if client_input.lower() in ["sair", "encerrar", "adeus", "obrigado"]:
            print(generate_farewell())
            break

        # Adiciona ao histórico (limitando o tamanho)
        conversation_history.append(f"Cliente: {client_input}")
        if len(conversation_history) > 6:  # Mantém apenas as últimas 3 interações
            conversation_history = conversation_history[-6:]

        # Obtém resposta
        response = call_groq_api(client_input, conversation_history)

        print("Assistente:", response)

        # Adiciona resposta ao histórico
        conversation_history.append(f"Assistente: {response}")

if __name__ == "__main__":
    main()

Boa tarde! Sou o assistente virtual da Neoenergia Elektro. Como posso ajudar você hoje?
Cliente: SAIR

Agradecemos seu contato! Sempre que precisar estamos disponíveis nos nossos canais:
💻 Site: https://www.neoenergia.com/web/sp

📱 Aplicativo:
Android → https://bit.ly/NELK-Android
IOS → https://bit.ly/IOS-NELK

📞 Teleatendimento: 116
Deficiente auditivo ou de fala | 0800 701 01 55
Atendimento fora da área de concessão | 0800 276 0116

Conte sempre com a gente!



**CÓDIGO CLAUDE**

In [2]:
!pip install python-dotenv

In [ ]:
import requests
import json
import datetime
import re
import os
from dotenv import load_dotenv

# Carregar variáveis de ambiente
load_dotenv()

# Configurações da API - Corrigindo o modo de acesso à API
API_KEY = os.getenv("GROQ_API_KEY", "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")  # Fallback para a chave fornecida
MODEL = "llama3-70b-8192"
URL = "https://api.groq.com/openai/v1/chat/completions"

# Base de conhecimento da Neoenergia Elektro
CATEGORIAS_PROBLEMAS = [
    {
        "categoria": "Técnico",
        "tipo": "EMERGENCIA MEDICA",
        "subtipo": "EQUIPAMENTO SUPORTE A VIDA",
        "descricao": "Falta de energia afetando equipamentos de suporte à vida",
        "causas": ["Falha na rede", "problema no fornecimento local"],
        "criticidade": "Emergência",
        "acao": "Atendimento imediato com prioridade máxima",
        "verificacoes": ["Verificar se há gerador auxiliar funcionando"],
        "termos": ["equipamento de suporte à vida", "respirador", "máquina de oxigênio"],
        "seguranca": "Acionar serviço médico de emergência simultaneamente",
        "interno": False
    },
    {
        "categoria": "Técnico",
        "tipo": "EMERGENCIA MEDICA",
        "subtipo": "HOSPITAL COM PROBLEMA",
        "descricao": "Falta de energia em hospital",
        "causas": ["Falha na rede", "problema no gerador"],
        "criticidade": "Emergência",
        "acao": "Atendimento imediato",
        "verificacoes": ["Verificar se o hospital tem gerador funcionando"],
        "termos": ["hospital está sem luz", "energia para um hospital"],
        "seguranca": "Solicitar prioridade ao informar a ocorrência",
        "interno": False
    },
    {
        "categoria": "Técnico",
        "tipo": "Falta de Energia",
        "subtipo": "Falta de Energia",
        "descricao": "Casa do cliente totalmente sem energia",
        "causas": ["Falha no ramal do cliente", "falha interna", "disjuntores desarmado",
                  "problema no medidor", "chave fusível desarmada"],
        "criticidade": "Médio",
        "acao": "Inspeção técnica e correção",
        "verificacoes": ["Conferir se disjuntores estão ligados", "testar outros aparelhos"],
        "termos": ["casa está sem luz", "só aqui em casa que caiu a energia"],
        "seguranca": "Desligar aparelhos eletrônicos",
        "interno": True
    },
    {
        "categoria": "Técnico",
        "tipo": "FALTA DE ENERGIA",
        "subtipo": "Falta de Energia em Medidor Eletrônico",
        "descricao": "Medidor eletrônico sem leitura ou funcionamento",
        "causas": ["Falha no medidor", "problema de comunicação", "bateria interna esgotada"],
        "criticidade": "Médio",
        "acao": "Inspeção técnica especializada",
        "verificacoes": ["Verificar se o display do medidor está aceso", "testar religação de disjuntores"],
        "termos": ["relógio digital está apagado", "medidor novo não está funcionando"],
        "seguranca": "Não tentar abrir ou manipular o medidor",
        "interno": False
    },
    {
        "categoria": "Técnico",
        "tipo": "Falta de Energia",
        "subtipo": "Falta de Energia Geral",
        "descricao": "Interrupção total de energia em uma área ampla",
        "causas": ["Falha em subestação", "curto-circuito na rede", "problema em transformador"],
        "criticidade": "Alto",
        "acao": "Manutenção corretiva urgente",
        "verificacoes": ["Verificar se vizinhos também estão sem luz"],
        "termos": ["faltou luz na rua", "vizinhança inteira sem energia", "apagão"],
        "seguranca": "Evitar usar velas, desligar aparelhos eletrônicos",
        "interno": False
    },
    {
        "categoria": "Técnico",
        "tipo": "FALTA DE ENERGIA",
        "subtipo": "FALTA DE FASE",
        "descricao": "Falta de uma das fases da rede elétrica, causando funcionamento parcial",
        "causas": ["Problema no transformador", "fusível queimado", "rompimento de um dos fios de fase"],
        "criticidade": "Médio",
        "acao": "Manutenção corretiva",
        "verificacoes": ["Testar se algumas tomadas funcionam e outras não"],
        "termos": ["algumas lâmpadas acendem, outras não", "só um lado da casa tem luz"],
        "seguranca": "Evitar o uso de aparelhos de alta potência",
        "interno": False
    },
    {
        "categoria": "Técnico",
        "tipo": "RISCO DE MORTE",
        "subtipo": "FIO PARTIDO NA REDE PRIMARIA",
        "descricao": "Cabo de alta tensão rompido",
        "causas": ["Acidente", "tempestade", "deterioração"],
        "criticidade": "Emergência",
        "acao": "Isolamento imediato da área e reparo urgente",
        "verificacoes": ["Observar cabo rompido em altura"],
        "termos": ["fio grosso da alta tensão está partido", "caiu um cabo de alta tensão"],
        "seguranca": "Manter distância mínima de 30 metros, alertar pessoas próximas",
        "interno": False
    },
    {
        "categoria": "Técnico",
        "tipo": "RISCO DE MORTE",
        "subtipo": "FIO PARTIDO NA REDE SECUNDARIA",
        "descricao": "Cabo de baixa tensão rompido",
        "causas": ["Acidente", "impacto", "deterioração"],
        "criticidade": "Emergência",
        "acao": "Isolamento da área e atendimento urgente",
        "verificacoes": ["Verificar cabo visivelmente partido"],
        "termos": ["fio da luz caiu", "cabo quebrado na rua"],
        "seguranca": "Manter distância mínima de 10 metros, alertar vizinhos",
        "interno": False
    },
    {
        "categoria": "Técnico",
        "tipo": "NIVEIS DE TENSÃO",
        "subtipo": "OSCILACÃO DE TENSÃO",
        "descricao": "Variações constantes na tensão elétrica",
        "causas": ["Sobrecarga momentânea", "falha no regulador", "problema no transformador"],
        "criticidade": "Médio",
        "acao": "Monitoramento e análise técnica",
        "verificacoes": ["Observar luzes piscando", "aparelhos reiniciando sozinhos"],
        "termos": ["luz fica piscando", "aparelhos desligam e ligam sozinhos"],
        "seguranca": "Desligar equipamentos sensíveis, utilizar estabilizadores",
        "interno": False
    },
    {
        "categoria": "Técnico",
        "tipo": "NIVEIS DE TENSÃO",
        "subtipo": "TENSÃO ALTA",
        "descricao": "Voltagem acima do normal, causando risco aos aparelhos",
        "causas": ["Regulador com defeito", "transformador sobrecarregado"],
        "criticidade": "Alto",
        "acao": "Inspeção técnica urgente",
        "verificacoes": ["Verificar se lâmpadas queimam com frequência ou brilham intensamente"],
        "termos": ["luz está muito forte", "aparelhos estão queimando"],
        "seguranca": "Desligar todos os aparelhos eletrônicos imediatamente",
        "interno": False
    },
    {
        "categoria": "Técnico",
        "tipo": "NIVEIS DE TENSÃO",
        "subtipo": "TENSÃO BAIXA",
        "descricao": "Voltagem abaixo do normal, afetando o funcionamento de aparelhos",
        "causas": ["Sobrecarga", "problema no transformador"],
        "criticidade": "Médio",
        "acao": "Inspeção técnica",
        "verificacoes": ["Testar se lâmpadas estão fracas e aparelhos falhando"],
        "termos": ["luz está fraca", "aparelhos não funcionam direito"],
        "seguranca": "Evitar ligar muitos aparelhos ao mesmo tempo",
        "interno": False
    },
    {
        "categoria": "Comercial",
        "tipo": "LIGAÇÃO",
        "subtipo": "Ligação Nova",
        "descricao": "Solicitação de ligação de energia para um novo imóvel",
        "causas": ["Novo cliente", "imóvel recém-construído"],
        "criticidade": "Médio",
        "acao": "Programação para instalação",
        "verificacoes": ["Verificar se há padrão de entrada instalado"],
        "termos": ["preciso de uma nova ligação de energia", "casa ainda não tem luz"],
        "seguranca": "Garantir que a instalação elétrica está regularizada antes da ligação",
        "interno": False
    },
    {
        "categoria": "Comercial",
        "tipo": "RELIGAÇÃO",
        "subtipo": "Religação Normal",
        "descricao": "Restabelecimento de energia após corte por inadimplência ou manutenção",
        "causas": ["Pagamento efetuado", "fim de manutenção programada"],
        "criticidade": "Médio",
        "acao": "Execução conforme prazo estabelecido",
        "verificacoes": ["Conferir se a conta foi paga e religação solicitada"],
        "termos": ["paguei a conta e a luz não voltou", "quero religar a energia"],
        "seguranca": "Esperar o prazo da distribuidora para religação antes de fazer nova solicitação",
        "interno": False
    }
]

def obter_saudacao():
    """Retorna uma saudação baseada no horário atual."""
    hora_atual = datetime.datetime.now().hour

    if 5 <= hora_atual < 12:
        return "Bom dia"
    elif 12 <= hora_atual < 18:
        return "Boa tarde"
    else:
        return "Boa noite"

def mensagem_fechamento():
    """Retorna a mensagem padrão de fechamento."""
    return """Agradecemos seu contato! Sempre que precisar estamos disponíveis nos nossos canais:
💻 Site: https://www.neoenergia.com/web/sp

📱 Aplicativo:
Android → https://bit.ly/NELK-Android
IOS → https://bit.ly/IOS-NELK

📞 Teleatendimento: 116
Deficiente auditivo ou de fala | 0800 701 01 55
Atendimento fora da área de concessão | 0800 276 0116

Conte sempre com a gente!"""

def analisar_problema_interno(mensagem):
    """
    Verifica se a mensagem indica um problema interno baseado em palavras-chave.
    Retorna (é_interno, dicas) onde:
    - é_interno: booleano indicando se é um problema interno
    - dicas: sugestões para verificação se for problema interno
    """
    # Palavras-chave que sugerem problemas internos
    padroes_internos = [
        r"só (em )?minha casa",
        r"(apenas|somente) (a )?minha casa",
        r"disjuntor( geral)? (desarm|caiu|desligou)",
        r"queim(ei|ou) um?a? (fusível|lâmpada)",
        r"geladeira (parou|não funciona)",
        r"(só |apenas |somente )(um cômodo|uma parte)",
        r"(já )?verifiqu?ei com (o vizinho|os vizinhos)",
        r"(não tem|sem) luz (apenas|só|somente) aqui"
    ]

    # Verificar padrões internos na mensagem
    for padrao in padroes_internos:
        if re.search(padrao, mensagem.lower()):
            return True, [
                "Verificar se os disjuntores estão ligados",
                "Testar se a energia funciona em algum ponto da casa",
                "Conferir se os vizinhos também estão sem energia"
            ]

    # Verificar correspondências com os problemas da base de conhecimento
    for problema in CATEGORIAS_PROBLEMAS:
        if problema.get("interno", False):
            for termo in problema.get("termos", []):
                if termo.lower() in mensagem.lower():
                    return True, problema.get("verificacoes", [])

    return False, []

def identificar_categoria_problema(mensagem):
    """
    Identifica a categoria de problema com base na mensagem do cliente.
    Retorna o problema mais relevante ou None se não encontrar.
    """
    relevancia_maxima = 0
    problema_identificado = None

    # Verificar criticidade emergência primeiro
    for problema in CATEGORIAS_PROBLEMAS:
        if problema.get("criticidade") == "Emergência":
            for termo in problema.get("termos", []):
                if termo.lower() in mensagem.lower():
                    return problema

    # Verificar demais problemas
    for problema in CATEGORIAS_PROBLEMAS:
        relevancia = 0
        for termo in problema.get("termos", []):
            if termo.lower() in mensagem.lower():
                relevancia += 1

        if relevancia > relevancia_maxima:
            relevancia_maxima = relevancia
            problema_identificado = problema

    return problema_identificado

def gerar_protocolo():
    """Gera um número de protocolo baseado na data e hora atuais."""
    agora = datetime.datetime.now()
    return f"{agora.year}{agora.month:02d}{agora.day:02d}{agora.hour:02d}{agora.minute:02d}{agora.second:02d}"

def consultar_ia(mensagem_cliente, contexto_anterior=None):
    """
    Consulta a IA para obter resposta ao cliente.

    Args:
        mensagem_cliente: Mensagem do cliente
        contexto_anterior: Histórico de mensagens anteriores (opcional)

    Returns:
        Resposta da IA
    """
    # Determinar se é problema interno
    e_interno, dicas_internas = analisar_problema_interno(mensagem_cliente)

    # Identificar categoria do problema
    problema = identificar_categoria_problema(mensagem_cliente)

    # Compilar informações para enviar à IA
    informacoes = {
        "mensagem_cliente": mensagem_cliente,
        "é_problema_interno": e_interno,
        "dicas_verificacao": dicas_internas,
        "problema_identificado": problema,
        "protocolo": gerar_protocolo() if not e_interno and problema else None
    }

    # Construir conteúdo do problema de forma segura
    problema_info = ""
    if problema:
        problema_info = f"""
        - Categoria: {problema.get('categoria', '')}
        - Tipo: {problema.get('tipo', '')}
        - Subtipo: {problema.get('subtipo', '')}
        - Descrição: {problema.get('descricao', '')}
        - Criticidade: {problema.get('criticidade', '')}
        - Ação necessária: {problema.get('acao', '')}
        - Cuidados de segurança: {problema.get('seguranca', '')}
        """

    # Criar prompt para a IA
    sistema_prompt = f"""
    Você é um atendente virtual da Neoenergia Elektro, uma distribuidora de energia.
    Seu nome é BotNinja e você deve atender ao cliente com empatia e objetividade.

    Informações sobre o problema do cliente:
    - Mensagem: {informacoes['mensagem_cliente']}
    - Problema interno: {"Sim" if informacoes['é_problema_interno'] else "Não"}

    {("- Dicas para verificação: " + ", ".join(informacoes['dicas_verificacao'])) if informacoes['dicas_verificacao'] else ""}

    {problema_info}

    {("- PROTOCOLO GERADO: " + informacoes['protocolo']) if informacoes['protocolo'] else ""}

    Diretrizes para sua resposta:
    1. Seja empático e compreensivo com o problema do cliente
    2. Use linguagem clara e objetiva, evitando textos muito longos
    3. Se for um problema interno, oriente o cliente a verificar os disjuntores e outras possíveis causas internas
    4. Se for um problema externo, informe o cliente sobre o protocolo gerado e os próximos passos
    5. Para problemas de Emergência ou Alto risco, enfatize as medidas de segurança
    6. Não use emojis excessivos nem linguagem muito técnica
    7. Não use "Neoenergia Elektro" em cada frase, apenas uma ou duas vezes na resposta
    8. Em sua resposta, você pode iniciar com "{obter_saudacao()}!"
    9. Não encerre a conversa a menos que o problema tenha sido resolvido
    """

    # Preparar headers e dados para a API
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }

    messages = [{"role": "system", "content": sistema_prompt}]

    # Adicionar contexto anterior se existir
    if contexto_anterior:
        messages.extend(contexto_anterior)

    # Adicionar mensagem atual do cliente
    messages.append({"role": "user", "content": mensagem_cliente})

    data = {
        "model": MODEL,
        "messages": messages,
        "temperature": 0.7,
        "max_tokens": 500
    }

    try:
        # Fazer requisição para a API - Adicionando mais informações de debug
        print(f"Enviando requisição para: {URL}")
        response = requests.post(URL, headers=headers, json=data)  # Usando json= em vez de data=json.dumps()

        if response.status_code != 200:
            print(f"Erro na API: Status {response.status_code}")
            print(f"Resposta: {response.text}")
            # Fallback para respostas em caso de erro na API
            return criar_resposta_fallback(e_interno, problema, informacoes)

        # Processar resposta
        resposta = response.json()
        conteudo = resposta["choices"][0]["message"]["content"]
        return conteudo

    except Exception as e:
        # Melhor tratamento de exceção com log do erro
        print(f"Erro ao chamar a API: {str(e)}")
        return criar_resposta_fallback(e_interno, problema, informacoes)

def criar_resposta_fallback(e_interno, problema, informacoes):
    """Cria uma resposta de fallback em caso de falha na API"""
    if e_interno:
        return f"{obter_saudacao()}! Identificamos que você pode estar com um problema na instalação interna. Recomendamos verificar os disjuntores e se os vizinhos também estão sem energia. Se o problema persistir, entre em contato novamente."
    elif problema and problema.get("criticidade") == "Emergência":
        return f"{obter_saudacao()}! Atenção! Identificamos uma situação de emergência. Por favor, siga as orientações de segurança: {problema.get('seguranca', 'Mantenha-se seguro e aguarde nossa equipe')}. Geramos o protocolo {informacoes['protocolo']} e uma equipe será enviada com urgência."
    elif problema:
        return f"{obter_saudacao()}! Registramos seu problema de {problema.get('descricao', 'energia')}. Geramos o protocolo {informacoes['protocolo']} para acompanhamento. Nossa equipe irá avaliar a situação o mais breve possível."
    else:
        return f"{obter_saudacao()}! Obrigado por entrar em contato com a Neoenergia Elektro. Para que possamos ajudar melhor, poderia fornecer mais detalhes sobre o problema que está enfrentando?"

def atendimento_cliente():
    """Função principal para iniciar atendimento a cliente."""
    print("\n=== Sistema de Atendimento Neoenergia Elektro ===\n")

    # Inicializar histórico de mensagens
    historico = []

    # Obter primeira mensagem do cliente
    print(f"[Atendente] {obter_saudacao()}! Sou o atendente virtual da Neoenergia Elektro. Como posso ajudar?")

    while True:
        mensagem_cliente = input("\n[Cliente] ")

        if mensagem_cliente.lower() in ["sair", "tchau", "encerrar", "finalizar"]:
            print(f"\n[Atendente] {mensagem_fechamento()}")
            break

        # Consultar IA para resposta
        resposta = consultar_ia(mensagem_cliente, historico)

        # Adicionar ao histórico
        historico.append({"role": "user", "content": mensagem_cliente})
        historico.append({"role": "assistant", "content": resposta})

        print(f"\n[Atendente] {resposta}")

def processar_mensagem_whatsapp(mensagem, numero_cliente, sessao_id=None):
    """
    Função para processar mensagens vindas do WhatsApp.

    Args:
        mensagem: Texto da mensagem do cliente
        numero_cliente: Número do WhatsApp do cliente
        sessao_id: ID da sessão para rastrear conversas (opcional)

    Returns:
        Resposta para enviar ao cliente
    """
    # Aqui implementaria a lógica para recuperar histórico da sessão
    historico = []  # Em produção, recuperaria do banco de dados

    # Consultar IA
    resposta = consultar_ia(mensagem, historico)

    # Salvar histórico (em produção)
    # salvar_historico(sessao_id, mensagem, resposta)

    return resposta

# Função para testar a conexão com a API
def testar_api():
    """Testa a conexão com a API para verificar se está funcionando"""
    print("Testando conexão com a API Groq...")
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }

    test_data = {
        "model": MODEL,
        "messages": [
            {"role": "system", "content": "Você é um assistente útil."},
            {"role": "user", "content": "Olá, está funcionando?"}
        ],
        "max_tokens": 50
    }

    try:
        response = requests.post(URL, headers=headers, json=test_data)
        if response.status_code == 200:
            print("✅ Conexão com a API estabelecida com sucesso!")
            return True
        else:
            print(f"❌ Erro na conexão: Status {response.status_code}")
            print(f"Resposta: {response.text}")
            return False
    except Exception as e:
        print(f"❌ Erro ao conectar à API: {str(e)}")
        return False

# Se executado diretamente (para teste)
if __name__ == "__main__":
    print("Iniciando sistema de atendimento Neoenergia Elektro...")
    print(f"Usando chave de API: {API_KEY[:5]}...{API_KEY[-4:]}")
    print(f"Modelo: {MODEL}")

    # Verificar se a API está funcionando
    if testar_api():
        atendimento_cliente()
    else:
        print("\nO sistema está operando no modo fallback devido a problemas na API.")
        print("As respostas serão limitadas, mas o atendimento continuará funcionando.")
        atendimento_cliente()

ModuleNotFoundError: No module named 'dotenv'

Código mais sucinto

In [ ]:
import requests
import json
import datetime
import re
import os
import pytz
from dotenv import load_dotenv

# Carregar variáveis de ambiente
load_dotenv()

# Configurações da API
API_KEY = os.getenv("GROQ_API_KEY", "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")
MODEL = "llama3-70b-8192"
URL = "https://api.groq.com/openai/v1/chat/completions"

# Configurar fuso horário de Brasília
TIMEZONE = pytz.timezone('America/Sao_Paulo')

# Base de conhecimento simplificada da Neoenergia Elektro
CATEGORIAS_PROBLEMAS = [
    {"nome": "Falta de energia geral", "criticidade": "Médio", "termos": ["sem luz", "energia caiu", "apagão"], "interno": False},
    {"nome": "Falta de fase", "criticidade": "Médio", "termos": ["falta de fase", "uma fase", "duas fases"], "interno": False},
    {"nome": "Tensão baixa", "criticidade": "Baixo", "termos": ["tensão baixa", "luz fraca"], "interno": False},
    {"nome": "Tensão alta", "criticidade": "Alto", "termos": ["tensão alta", "luz forte", "queimando"], "interno": False},
    {"nome": "Fio partido", "criticidade": "Emergência", "termos": ["fio partido", "cabo rompido", "fio caído"], "seguranca": "Mantenha distância do fio!", "interno": False},
    {"nome": "Fogo na rede", "criticidade": "Emergência", "termos": ["fogo na rede", "fio pegando fogo", "faísca"], "seguranca": "Afaste-se do local e chame os bombeiros!", "interno": False},
    {"nome": "Choque elétrico", "criticidade": "Emergência", "termos": ["choque", "eletrocutado", "levando choque"], "seguranca": "Desligue a energia imediatamente!", "interno": False},
    {"nome": "Disjuntor desarmado", "criticidade": "Baixo", "termos": ["disjuntor desarmou", "disjuntor caiu"], "verificacoes": ["Verifique o disjuntor", "Religue o disjuntor"], "interno": True},
    {"nome": "Problema no medidor", "criticidade": "Médio", "termos": ["medidor parado", "relógio de luz não funciona"], "interno": False},
    {"nome": "Religação de energia", "criticidade": "Médio", "termos": ["religar energia", "voltar a luz", "religação"], "interno": False},
    {"nome": "Ligação nova", "criticidade": "Médio", "termos": ["nova ligação", "primeira vez", "instalação nova"], "interno": False}
]

def obter_saudacao():
    """Retorna uma saudação baseada no horário de Brasília."""
    try:
        hora_atual = datetime.datetime.now(TIMEZONE).hour
        if 5 <= hora_atual < 12:
            return "Bom dia"
        elif 12 <= hora_atual < 18:
            return "Boa tarde"
        else:
            return "Boa noite"
    except Exception:
        return "Olá"

def mensagem_fechamento():
    """Retorna a mensagem padrão de fechamento."""
    return "Agradecemos seu contato! Para mais informações: Site neoenergia.com ou Teleatendimento 116"

def analisar_problema_interno(mensagem):
    """Verifica se a mensagem indica um problema interno."""
    mensagem = mensagem.lower()
    for problema in CATEGORIAS_PROBLEMAS:
        if problema.get("interno", False):
            for termo in problema.get("termos", []):
                if termo.lower() in mensagem:
                    return True, problema.get("verificacoes", [])
    return False, []

def identificar_categoria_problema(mensagem):
    """Identifica a categoria de problema com base na mensagem."""
    mensagem = mensagem.lower()
    # Verificar emergências primeiro
    for problema in CATEGORIAS_PROBLEMAS:
        if problema.get("criticidade") == "Emergência":
            for termo in problema.get("termos", []):
                if termo.lower() in mensagem:
                    return problema
    # Verificar outros problemas
    for problema in CATEGORIAS_PROBLEMAS:
        for termo in problema.get("termos", []):
            if termo.lower() in mensagem:
                return problema
    return None

def gerar_protocolo():
    """Gera um número de protocolo baseado na data e hora atuais em Brasília."""
    agora = datetime.datetime.now(TIMEZONE)
    return f"{agora.year}{agora.month:02d}{agora.day:02d}{agora.hour:02d}{agora.minute:02d}{agora.second:02d}"

def consultar_ia(mensagem_cliente, contexto_anterior=None):
    """Consulta a IA para obter resposta ao cliente."""
    e_interno, dicas_internas = analisar_problema_interno(mensagem_cliente)
    problema = identificar_categoria_problema(mensagem_cliente)
    protocolo = gerar_protocolo() if not e_interno and problema else None

    sistema_prompt = f"""
    Você é o atendente virtual da Neoenergia Elektro. Seja objetivo e claro.

    Informações:
    - Mensagem: {mensagem_cliente}
    - Problema interno: {"Sim" if e_interno else "Não"}
    {f"- Dicas: {', '.join(dicas_internas)}" if dicas_internas else ""}
    {f"- Tipo de problema: {problema['nome']}" if problema else ""}
    {f"- Criticidade: {problema['criticidade']}" if problema else ""}
    {f"- Protocolo: {protocolo}" if protocolo else ""}

    Regras:
    1. Respostas curtas (1-2 frases)
    2. Para problemas internos: dê apenas 1 verificação
    3. Emergências: comece com "ATENÇÃO!" e inclua segurança
    4. Use linguagem simples
    5. Quando gerar protocolo, SEMPRE informe o tipo de problema
    6. Formato para protocolos: "[Tipo de problema] - Protocolo: [número]"
    """

    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }

    messages = [{"role": "system", "content": sistema_prompt}]
    if contexto_anterior:
        messages.extend(contexto_anterior)
    messages.append({"role": "user", "content": mensagem_cliente})

    data = {
        "model": MODEL,
        "messages": messages,
        "temperature": 0.7,
        "max_tokens": 100
    }

    try:
        response = requests.post(URL, headers=headers, json=data)
        return response.json()["choices"][0]["message"]["content"]
    except Exception:
        return criar_resposta_fallback(e_interno, problema, protocolo)

def criar_resposta_fallback(e_interno, problema, protocolo):
    """Resposta de fallback mais curta"""
    if e_interno:
        return "Verifique os disjuntores. Se persistir, nos avise."
    elif problema and problema.get("criticidade") == "Emergência":
        return f"ATENÇÃO! {problema.get('seguranca', 'Mantenha-se afastado!')}\n[{problema['nome']}] - Protocolo: {protocolo}"
    elif problema:
        return f"[{problema['nome']}] - Protocolo: {protocolo}. Iremos resolver."
    else:
        return "Poderia detalhar melhor o problema?"

def atendimento_cliente():
    """Função principal para iniciar atendimento."""
    print("\n=== Atendimento Neoenergia Elektro ===\n")


    historico = []
    primeira_mensagem = True

    while True:
        if primeira_mensagem:
            print(f"[Atendente] {obter_saudacao()}! Como posso ajudar?")
            primeira_mensagem = False

        mensagem_cliente = input("\n[Cliente] ").strip()

        if mensagem_cliente.lower() in ["sair", "tchau", "encerrar", "obrigado", "agradeço"]:
            print(f"\n[Atendente] {mensagem_fechamento()}")
            break

        resposta = consultar_ia(mensagem_cliente, historico)
        historico.append({"role": "user", "content": mensagem_cliente})
        historico.append({"role": "assistant", "content": resposta})

        print(f"\n[Atendente] {resposta}")

if __name__ == "__main__":
    atendimento_cliente()


=== Atendimento Neoenergia Elektro ===

[Atendente] Boa tarde! Como posso ajudar?

[Cliente] as lampadas estão piscando aqui em casa

[Atendente] Vou ajudá-lo! Isso pode ser um problema de fornecimento de energia. Vou gerar um protocolo para você. "Problema de Fornecimento de Energia - Protocolo: 001". Aguarde um momento, por favor.


Versão Final
Versão Final
Versão Final
Versão Final
Versão Final
Versão Final
Versão Final
Versão Final
Versão Final

In [ ]:
import requests
import json
import datetime
import re
import os
import pytz
from dotenv import load_dotenv

# Carregar variáveis de ambiente
load_dotenv()

# Configurações da API
API_KEY = os.getenv("GROQ_API_KEY", "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")
MODEL = "llama3-70b-8192"
URL = "https://api.groq.com/openai/v1/chat/completions"

# Configurar fuso horário de Brasília
TIMEZONE = pytz.timezone('America/Sao_Paulo')

# Base de conhecimento simplificada da Neoenergia Elektro
CATEGORIAS_PROBLEMAS = [
    {"categoria": "Ocorrência Técnica", "tipo": "Emergencia médica", "subtipo": "EMERGENCIA MEDICA - EQUIPAMENTO SUPORTE A VIDA", "criticidade": "Emergência", "termos": ["Tenho equipamento de suporte à vida", "Estou com respirador", "Minha máquina de oxigênio parou"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Emergencia médica", "subtipo": "EMERGENCIA MEDICA - HOSPITAL COM PROBLEMA", "criticidade": "Emergência", "termos": ["O hospital está sem luz", "Preciso de energia para um hospital"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Falta de Energia", "subtipo": "Falta de Energia - Geral", "criticidade": "Médio", "termos": ["Minha casa está sem luz", "Só aqui em casa que caiu a energia"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Falta de Energia", "subtipo": "Falta de Energia em Medidor Eletrônico", "criticidade": "Médio", "termos": ["Meu relógio digital está apagado", "O medidor novo não está funcionando"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Falta de Energia", "subtipo": "Falta de Energia Geral - Área Amplia", "criticidade": "Alto", "termos": ["Faltou luz na rua", "A vizinhança inteira sem energia", "Apagão"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Falta de Energia", "subtipo": "FALTA DE FASE", "criticidade": "Médio", "termos": ["Algumas lâmpadas acendem, outras não", "Só um lado da casa tem luz"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Falta de Energia", "subtipo": "Falta de Fase em Medidor Eletrônico", "criticidade": "Médio", "termos": ["Meu medidor está piscando em vermelho", "Aparece um erro no painel do relógio"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Falta de Energia", "subtipo": "SIGFI FALTA DE ENERGIA FOTOVOLTAICO", "criticidade": "Médio", "termos": ["Meu sistema solar não está gerando energia", "As baterias não estão carregando"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Níveis de tensão", "subtipo": "OSCILACÃO DE TENSÃO", "criticidade": "Médio", "termos": ["A luz fica piscando", "Meus aparelhos desligam e ligam sozinhos"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Níveis de tensão", "subtipo": "TENSÃO ALTA", "criticidade": "Alto", "termos": ["A luz está muito forte", "Meus aparelhos estão queimando"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Níveis de tensão", "subtipo": "TENSÃO BAIXA", "criticidade": "Médio", "termos": ["A luz está fraca", "Meus aparelhos não funcionam direito"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Outros", "subtipo": "TRANSFORMADOR", "criticidade": "Alto", "termos": ["O transformador está fazendo barulho", "Tem um cheiro estranho vindo do poste"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "CHOQUE EM INSTALAÇÃO", "criticidade": "Emergência", "termos": ["Estou tomando choque na torneira", "A geladeira está dando choque"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "ELO FUSIVEL QUEIMADO/CAIDO", "criticidade": "Alto", "termos": ["Tem algo pendurado no poste", "Um dispositivo caiu do transformador"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "FALHA EM MEDIDOR", "criticidade": "Médio", "termos": ["Meu relógio de luz está com problema", "O medidor parece quebrado"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "FALHA EM QUICK-LAG", "criticidade": "Médio", "termos": ["O disjuntor do poste parece queimado", "Algo explodiu na caixa do poste"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "FALHA EM RAMAL DE SERVICO", "criticidade": "Alto", "termos": ["O fio que vem para minha casa está solto", "A conexão com o poste está com problema"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "FIO DANDO FOGO COM ARVORE NA REDE", "criticidade": "Emergência", "termos": ["Está saindo faísca do fio com a árvore", "A árvore está pegando fogo no fio"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "FIO DANDO FOGO NA REDE PRIMARIA", "criticidade": "Emergência", "termos": ["Está saindo fogo dos fios de alta tensão", "Os cabos grossos estão soltando faíscas"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "FIO DANDO FOGO NA REDE SECUNDARIA", "criticidade": "Alto", "termos": ["O fio da rua está soltando faísca", "Está saindo fogo do cabo de luz"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "FIO DANDO FOGO NO RAMAL DE SERVICO", "criticidade": "Alto", "termos": ["O fio que entra na minha casa está pegando fogo", "Tem faísca na entrada de luz"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "FURTO EQUIPAMENTO", "criticidade": "Alto", "termos": ["Roubaram algo do poste", "Levaram equipamento da rede"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "FURTO FIACÃO/PADRÃO", "criticidade": "Alto", "termos": ["Roubaram os fios da minha casa", "Levaram o medidor"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "FURTO REDE", "criticidade": "Alto", "termos": ["Roubaram os fios da rua", "Tem cabos cortados no poste"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "INVERSÃO DE FASE", "criticidade": "Médio", "termos": ["Meu motor está girando ao contrário", "O equipamento trifásico não funciona direito"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "OBJETO ESTRANHO NA REDE PRIMARIA", "criticidade": "Alto", "termos": ["Tem algo preso nos fios de alta tensão", "Um objeto está encostado na rede"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "OBJETO ESTRANHO NA REDE SECUNDARIA", "criticidade": "Médio", "termos": ["Jogaram algo nos fios", "Tem um objeto preso na fiação"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "POSTE PADRÃO QUEBRADO", "criticidade": "Alto", "termos": ["Meu poste padrão está quebrado", "A entrada de luz está torta"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Risco de Morte", "subtipo": "FIO PARTIDO NA REDE PRIMARIA", "criticidade": "Emergência", "termos": ["O fio grosso da alta tensão está partido", "Caiu um cabo de alta tensão"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Risco de Morte", "subtipo": "FIO PARTIDO NA REDE SECUNDARIA", "criticidade": "Emergência", "termos": ["O fio da luz caiu", "Tem um cabo quebrado na rua"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Risco de Morte", "subtipo": "FIO PARTIDO NO RAMAL DE SERVICO", "criticidade": "Alto", "termos": ["O fio que vem para minha casa está quebrado", "Caiu o cabo da entrada"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Risco de Morte", "subtipo": "POSTE ABALROADO COM FIO PARTIDO", "criticidade": "Emergência", "termos": ["Alguém bateu no poste e o fio caiu", "Poste quebrado com fios soltos"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Risco de Morte", "subtipo": "POSTE ABALROADO SEM FIO PARTIDO", "criticidade": "Alto", "termos": ["O poste está torto depois do acidente", "Bateram no poste mas os fios estão intactos"], "interno": False},
    {"categoria": "Ocorrência Comercial", "tipo": "Ligação", "subtipo": "Ligação Nova", "criticidade": "Médio", "termos": ["Preciso de uma nova ligação de energia", "Minha casa ainda não tem luz"], "interno": False},
    {"categoria": "Ocorrência Comercial", "tipo": "Religação", "subtipo": "Religação Normal", "criticidade": "Médio", "termos": ["Paguei a conta e a luz não voltou", "Quero religar a energia"], "interno": False},
    {"categoria": "Ocorrência Comercial", "tipo": "Religação", "subtipo": "Religação de Urgência", "criticidade": "Alto", "termos": ["Preciso da energia hoje mesmo", "Tenho uma pessoa doente em casa"], "interno": False},
    {"categoria": "Ocorrência Comercial", "tipo": "Desligamento progamado", "subtipo": "Desligamento para Manutenção", "criticidade": "Médio", "termos": ["Vai faltar luz aqui?", "Tem alguma manutenção programada?"], "interno": False}
]

def obter_saudacao():
    """Retorna uma saudação baseada no horário de Brasília."""
    try:
        hora_atual = datetime.datetime.now(TIMEZONE).hour
        if 5 <= hora_atual < 12:
            return "Bom dia"
        elif 12 <= hora_atual < 18:
            return "Boa tarde"
        else:
            return "Boa noite"
    except Exception:
        return "Olá"

def mensagem_fechamento():
    """Retorna a mensagem padrão de fechamento."""
    return """Agradecemos seu contato! Sempre que precisar estamos disponíveis nos nossos canais:

💻 Site: https://www.neoenergia.com/web/sp

📱 Aplicativo:
Android → https://bit.ly/NELK-Android
IOS → https://bit.ly/IOS-NELK

📞 Teleatendimento: 116
Deficiente auditivo ou de fala | 0800 701 01 55
Atendimento fora da área de concessão | 0800 276 0116
Conte sempre com a gente!"""

def analisar_problema_interno(mensagem):
    """Verifica se a mensagem indica um problema interno."""
    mensagem = mensagem.lower()
    for problema in CATEGORIAS_PROBLEMAS:
        if problema.get("interno", False):
            for termo in problema.get("termos", []):
                if termo.lower() in mensagem:
                    return True, problema.get("verificacoes", [])
    return False, []

def identificar_categoria_problema(mensagem):
    """Identifica a categoria de problema com base na mensagem."""
    mensagem = mensagem.lower()
    # Verificar emergências primeiro
    for problema in CATEGORIAS_PROBLEMAS:
        if problema.get("criticidade") == "Emergência":
            for termo in problema.get("termos", []):
                if termo.lower() in mensagem:
                    return problema
    # Verificar outros problemas
    for problema in CATEGORIAS_PROBLEMAS:
        for termo in problema.get("termos", []):
            if termo.lower() in mensagem:
                return problema
    return None

def gerar_protocolo():
    """Gera um número de protocolo baseado na data e hora atuais em Brasília."""
    agora = datetime.datetime.now(TIMEZONE)
    return f"{agora.year}{agora.month:02d}{agora.day:02d}{agora.hour:02d}{agora.minute:02d}{agora.second:02d}"

def consultar_ia(mensagem_cliente, contexto_anterior=None):
    """Consulta a IA para obter resposta ao cliente."""
    e_interno, dicas_internas = analisar_problema_interno(mensagem_cliente)
    problema = identificar_categoria_problema(mensagem_cliente)
    protocolo = gerar_protocolo() if not e_interno and problema else None

    sistema_prompt = f"""
    Você é o atendente virtual da Neoenergia Elektro. Seja objetivo e claro.

    Informações:
    - Mensagem: {mensagem_cliente}
    - Problema interno: {"Sim" if e_interno else "Não"}
    {f"- Dicas: {', '.join(dicas_internas)}" if dicas_internas else ""}
    {f"- Categoria: {problema['categoria']}" if problema else ""}
    {f"- Tipo de problema: {problema['tipo']}" if problema else ""}
    {f"- Subtipo: {problema['subtipo']}" if problema else ""}
    {f"- Criticidade: {problema['criticidade']}" if problema else ""}
    {f"- Protocolo: {protocolo}" if protocolo else ""}

    Regras:
    1. Primeiro identifique claramente o tipo e subtipo do problema
    2. Confirme com o cliente se ele quer abrir um chamado técnico
    3. Só depois gere o protocolo quando confirmado
    4. Formato para identificação: "Isso parece ser um problema de [Tipo] - [Subtipo]"
    5. Formato para confirmação: "Você quer que eu abra um chamado técnico para [Subtipo]?"
    6. Formato para protocolo: "Seu protocolo é [número]. Nossa equipe irá resolver."
    7. Emergências: comece com "ATENÇÃO!" e inclua instruções de segurança
    8. Mantenha respostas curtas e objetivas
    """

    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }

    messages = [{"role": "system", "content": sistema_prompt}]
    if contexto_anterior:
        messages.extend(contexto_anterior)
    messages.append({"role": "user", "content": mensagem_cliente})

    data = {
        "model": MODEL,
        "messages": messages,
        "temperature": 0.7,
        "max_tokens": 150
    }

    try:
        response = requests.post(URL, headers=headers, json=data)
        return response.json()["choices"][0]["message"]["content"]
    except Exception:
        return criar_resposta_fallback(e_interno, problema, protocolo)

def criar_resposta_fallback(e_interno, problema, protocolo):
    """Resposta de fallback mais curta"""
    if e_interno:
        return "Verifique os disjuntores. Se persistir, nos avise."
    elif problema:
        if problema.get("criticidade") == "Emergência":
            return f"ATENÇÃO! {problema.get('seguranca', 'Mantenha-se afastado!')}\nIsso parece ser um problema de {problema['tipo']}, pelo que me disse acredito que seja {problema['subtipo']}. Você quer que eu abra um chamado técnico?"
        else:
            return f"Isso parece ser um problema de {problema['tipo']} - {problema['subtipo']}. Você quer que eu abra um chamado técnico?"
    else:
        return "Poderia detalhar melhor o problema?"

def atendimento_cliente():
    """Função principal para iniciar atendimento."""
    print("\n=== Atendimento Neoenergia Elektro ===\n")


    historico = []
    primeira_mensagem = True

    while True:
        if primeira_mensagem:
            print(f"[Atendente] {obter_saudacao()}! Como posso ajudar?")
            primeira_mensagem = False

        mensagem_cliente = input("\n[Cliente] ").strip()

        if mensagem_cliente.lower() in ["sair", "tchau", "encerrar", "obrigado", "agradeço"]:
            print(f"\n[Atendente] {mensagem_fechamento()}")
            break

        resposta = consultar_ia(mensagem_cliente, historico)
        historico.append({"role": "user", "content": mensagem_cliente})
        historico.append({"role": "assistant", "content": resposta})

        print(f"\n[Atendente] {resposta}")

if __name__ == "__main__":
    atendimento_cliente()


=== Atendimento Neoenergia Elektro ===

[Atendente] Bom dia! Como posso ajudar?

[Cliente] estou com um problema aqui em casa, o meu chuveiro não está ligando

[Atendente] Isso parece ser um problema de Equipamento - Chuveiro.

Você quer que eu abra um chamado técnico para verificar o problema do seu chuveiro?
